# Neural Style

Lab created by Daniel Fojo for the UPC school 2020

# Hyperparameters

In [ ]:
style_weight = 5e3
content_weight = 1
variation_weight = 100

# Imports

In [ ]:
import torch
from torchvision import models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda")


In this session we will use the approach described in [this paper](https://arxiv.org/abs/1508.06576) to apply a particular style to a content image by means of optimization using the weights of a pretrained convnet (VGG16).

# Download images

Let's first display the images we chose. We want to transfer the style of the second image into the first one.

In [ ]:
from PIL import Image
# We get the images from the following repository
!wget https://cdn.archpaper.com/wp-content/uploads/2020/06/maarten-van-den-heuvel-gZXx8lKAb7Y-unsplash.jpg -O base.jpg
!wget https://images.fineartamerica.com/images/artworkimages/mediumlarge/3/starry-night-print-by-vincent-van-gogh-vincent-van-gogh.jpg -O style.jpg
base_image_path = 'base.jpg'
style_reference_image_path = 'style.jpg'

base_image = Image.open(base_image_path)
style_reference_image = Image.open(style_reference_image_path)


In [ ]:
f, axarr = plt.subplots(1,2,figsize=(20,20))
axarr[0].imshow(np.asarray(base_image))
axarr[0].axis('off')
axarr[1].imshow(np.asarray(style_reference_image))
axarr[1].axis('off')
plt.show()


# Create model

Load VGG and get a dictionary of all its layers:

In [ ]:
import torchvision.models as models

model = models.vgg16(pretrained=True).features.to(device).eval()
model

Here we choose the layers of the network that we want to use to represent the style and the content. You can play with different combinations if you wish.

In [ ]:
content_layers = [20]

style_layers = [0, 5, 10, 17, 24]

## Exercise 1
Complete the code to have a FeaturesExtractor module that returns 2 lists, one with the content and one with the style features.

**Hint:** You can use `for i, layer in enumerate(self.base_model):` to iterate the layers of the model in order one by one.

In [ ]:
import torch.nn as nn
class FeaturesExtractor(nn.Module):

    def __init__(self, base_model, content_layers, style_layers):
        super().__init__()
        self.base_model = base_model
        self.content_layers = content_layers
        self.style_layers = style_layers

    def forward(self, x):
        content_output = []
        style_output = []
        ...
        return content_output, style_output


We don't need to compute the gradients of this model.

In [ ]:
feature_extractor = FeaturesExtractor(model, content_layers, style_layers)
for param in feature_extractor.parameters():
    param.requires_grad = False


# Loss functions

## Exercise 2
Complete the content loss and the style loss.

**Hint:** Remember that the content loss is Mean Square Error for the features, and the style loss is Mean Square Error for the Gram matrices of the features.

In [ ]:
content_loss = ...

def gram_matrix(x):
    b, c, h, w = x.shape
    x = x.reshape(b, c, h*w)
    gram = torch.bmm(x, x.transpose(1, 2))/(h*w)
    return gram

def style_loss(style, combination):
    ...
    return F.mse_loss(S, C)


## Extra: Total variation loss

Adding the total variation loss to the training loop can help a bit in maintaining local coherence.

In [ ]:
def total_variation_loss(img):      
    tv_h = ((img[:,:,1:,:] - img[:,:,:-1,:]).pow(2)).mean()
    tv_w = ((img[:,:,:,1:] - img[:,:,:,:-1]).pow(2)).mean()    
    return tv_h + tv_w

# Prepare data

We will now prepare the data for the model.

In [ ]:
img_size = 512

transform = transforms.Compose([
                                transforms.Resize(img_size),
                                transforms.CenterCrop(img_size),
                                transforms.ToTensor(),
])

base_tensor = transform(base_image).to(device).unsqueeze(0)
style_reference_tensor = transform(style_reference_image).to(device).unsqueeze(0)


## Exercise 3
Create the `combination_tensor` with the same initial values as base_tensor. It will be the image that we will "train". Then, use `optim.LBFGS` as an optimizer to train it, with the default value for lr.

**Hint:** You can copy another tensor by using `.clone()`. Remember that trainable tensors have to have `requires_grad=True`

In [ ]:
combination_tensor = ...
...

optimizer = ...


# Extract target features

In [ ]:
content_target, _ = feature_extractor(base_tensor)
_, style_target = feature_extractor(style_reference_tensor)
content_target = [ct.detach() for ct in content_target]
style_target = [st.detach() for st in style_target]


# Training loop

## Exercise 4

Complete the training loop. 

Take into account that the optimizer LBFGS requires a closure, which is a function that sets the gradients to 0, evaluates the model with the current data, computes the loss, computes the gradients and then returns the current loss. Remember to multiply each loss times the corresponding weight defined in the hyperparameters cell.



In [ ]:

images_history = []

for i in range(iterations):

    def closure():
        # Reset gradients
        ...
        loss = 0.0

        content_output, style_output = feature_extractor(combination_tensor)

        # Content loss
        cl = 0
        for orignal_feature, combination_feature in zip(content_target, content_output):
            cl += ...
        loss += cl / len(content_layers)
        
        # Style loss
        sl = 0
        for orignal_feature, combination_feature in zip(style_target, style_output):
            sl += ...
        loss +=  sl / len(style_layers)

        # Variation loss
        vl = ...
        loss += vl

        if i % 2 == 0:
            print(f"[{i}] Content loss: {cl.item():.2f}, Style loss: {sl.item():.2f}, Variation loss: {vl.item():.2f}")

        # Compute gradients
        ...
        return loss
    
    optimizer.step(closure)
    images_history.append(combination_tensor.clone().cpu().squeeze().detach().clamp(0, 1))


In [ ]:
images_history = [transforms.functional.to_pil_image(im) for im in images_history]


In [ ]:
images_history[-1]
